In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 21: Keypad Conundrum ---</h2><p>As you teleport onto Santa's <a href="/2019/day/25">Reindeer-class starship</a>, The Historians begin to panic: someone from their search party is <em>missing</em>. A quick life-form scan by the ship's computer reveals that when the missing Historian teleported, he arrived in another part of the ship.</p>
<p>The door to that area is locked, but the computer can't open it; it can only be opened by <em>physically typing</em> the door codes (your puzzle input) on the numeric keypad on the door.</p>
<p>The numeric keypad has four rows of buttons: <code>789</code>, <code>456</code>, <code>123</code>, and finally an empty gap followed by <code>0A</code>. Visually, they are arranged like this:</p>
<pre><code>+---+---+---+
| 7 | 8 | 9 |
+---+---+---+
| 4 | 5 | 6 |
+---+---+---+
| 1 | 2 | 3 |
+---+---+---+
    | 0 | A |
    +---+---+
</code></pre>
<p>Unfortunately, the area outside the door is currently <em>depressurized</em> and nobody can go near the door. A robot needs to be sent instead.</p>
<p>The robot has no problem navigating the ship and finding the numeric keypad, but it's not designed for button pushing: it can't be told to push a specific button directly. Instead, it has a robotic arm that can be controlled remotely via a <em>directional keypad</em>.</p>
<p>The directional keypad has two rows of buttons: a gap / <code>^</code> (up) / <code>A</code> (activate) on the first row and <code>&lt;</code> (left) / <code>v</code> (down) / <code>&gt;</code> (right) on the second row. Visually, they are arranged like this:</p>
<pre><code>    +---+---+
    | ^ | A |
+---+---+---+
| &lt; | v | &gt; |
+---+---+---+
</code></pre>
<p>When the robot arrives at the numeric keypad, its robotic arm is pointed at the <code>A</code> button in the bottom right corner. After that, this directional keypad remote control must be used to maneuver the robotic arm: the up / down / left / right buttons cause it to move its arm one button in that direction, and the <code>A</code> button causes the robot to briefly move forward, pressing the button being aimed at by the robotic arm.</p>
<p>For example, to make the robot type <code>029A</code> on the numeric keypad, one sequence of inputs on the directional keypad you could use is:</p>
<ul>
<li><code>&lt;</code> to move the arm from <code>A</code> (its initial position) to <code>0</code>.</li>
<li><code>A</code> to push the <code>0</code> button.</li>
<li><code>^A</code> to move the arm to the <code>2</code> button and push it.</li>
<li><code>&gt;^^A</code> to move the arm to the <code>9</code> button and push it.</li>
<li><code>vvvA</code> to move the arm to the <code>A</code> button and push it.</li>
</ul>
<p>In total, there are three shortest possible sequences of button presses on this directional keypad that would cause the robot to type <code>029A</code>: <code>&lt;A^A&gt;^^AvvvA</code>, <code>&lt;A^A^&gt;^AvvvA</code>, and <code>&lt;A^A^^&gt;AvvvA</code>.</p>
<p>Unfortunately, the area containing this directional keypad remote control is currently experiencing <em>high levels of radiation</em> and nobody can go near it. A robot needs to be sent instead.</p>
<p>When the robot arrives at the directional keypad, its robot arm is pointed at the <code>A</code> button in the upper right corner. After that, a <em>second, different</em> directional keypad remote control is used to control this robot (in the same way as the first robot, except that this one is typing on a directional keypad instead of a numeric keypad).</p>
<p>There are multiple shortest possible sequences of directional keypad button presses that would cause this robot to tell the first robot to type <code>029A</code> on the door. One such sequence is <code>v&lt;&lt;A&gt;&gt;^A&lt;A&gt;AvA&lt;^AA&gt;A&lt;vAAA&gt;^A</code>.</p>
<p>Unfortunately, the area containing this second directional keypad remote control is currently <em><code>-40</code> degrees</em>! Another robot will need to be sent to type on that directional keypad, too.</p>
<p>There are many shortest possible sequences of directional keypad button presses that would cause this robot to tell the second robot to tell the first robot to eventually type <code>029A</code> on the door. One such sequence is <code>&lt;vA&lt;AA&gt;&gt;^AvAA&lt;^A&gt;A&lt;v&lt;A&gt;&gt;^AvA^A&lt;vA&gt;^A&lt;v&lt;A&gt;^A&gt;AAvA^A&lt;v&lt;A&gt;A&gt;^AAAvA&lt;^A&gt;A</code>.</p>
<p>Unfortunately, the area containing this third directional keypad remote control is currently <em>full of Historians</em>, so no robots can find a clear path there. Instead, <em>you</em> will have to type this sequence yourself.</p>
<p>Were you to choose this sequence of button presses, here are all of the buttons that would be pressed on your directional keypad, the two robots' directional keypads, and the numeric keypad:</p>
<pre><code>&lt;vA&lt;AA&gt;&gt;^AvAA&lt;^A&gt;A&lt;v&lt;A&gt;&gt;^AvA^A&lt;vA&gt;^A&lt;v&lt;A&gt;^A&gt;AAvA^A&lt;v&lt;A&gt;A&gt;^AAAvA&lt;^A&gt;A
v&lt;&lt;A&gt;&gt;^A&lt;A&gt;AvA&lt;^AA&gt;A&lt;vAAA&gt;^A
&lt;A^A&gt;^^AvvvA
029A
</code></pre>
<p>In summary, there are the following keypads:</p>
<ul>
<li>One directional keypad that <em>you</em> are using.</li>
<li>Two directional keypads that <em>robots</em> are using.</li>
<li>One numeric keypad (on a door) that a <em>robot</em> is using.</li>
</ul>
<p>It is important to remember that these robots are not designed for button pushing. In particular, if a robot arm is ever aimed at a <em>gap</em> where no button is present on the keypad, even for an instant, the robot will <em>panic</em> unrecoverably. So, don't do that. All robots will initially aim at the keypad's <code>A</code> key, wherever it is.</p>
<p>To unlock the door, <em>five</em> codes will need to be typed on its numeric keypad. For example:</p>
<pre><code>029A
980A
179A
456A
379A
</code></pre>
<p>For each of these, here is a shortest sequence of button presses you could type to cause the desired code to be typed on the numeric keypad:</p>
<pre><code>029A: &lt;vA&lt;AA&gt;&gt;^AvAA&lt;^A&gt;A&lt;v&lt;A&gt;&gt;^AvA^A&lt;vA&gt;^A&lt;v&lt;A&gt;^A&gt;AAvA^A&lt;v&lt;A&gt;A&gt;^AAAvA&lt;^A&gt;A
980A: &lt;v&lt;A&gt;&gt;^AAAvA^A&lt;vA&lt;AA&gt;&gt;^AvAA&lt;^A&gt;A&lt;v&lt;A&gt;A&gt;^AAAvA&lt;^A&gt;A&lt;vA&gt;^A&lt;A&gt;A
179A: &lt;v&lt;A&gt;&gt;^A&lt;vA&lt;A&gt;&gt;^AAvAA&lt;^A&gt;A&lt;v&lt;A&gt;&gt;^AAvA^A&lt;vA&gt;^AA&lt;A&gt;A&lt;v&lt;A&gt;A&gt;^AAAvA&lt;^A&gt;A
456A: &lt;v&lt;A&gt;&gt;^AA&lt;vA&lt;A&gt;&gt;^AAvAA&lt;^A&gt;A&lt;vA&gt;^A&lt;A&gt;A&lt;vA&gt;^A&lt;A&gt;A&lt;v&lt;A&gt;A&gt;^AAvA&lt;^A&gt;A
379A: &lt;v&lt;A&gt;&gt;^AvA^A&lt;vA&lt;AA&gt;&gt;^AAvA&lt;^A&gt;AAvA^A&lt;vA&gt;^AA&lt;A&gt;A&lt;v&lt;A&gt;A&gt;^AAAvA&lt;^A&gt;A
</code></pre>
<p>The Historians are getting nervous; the ship computer doesn't remember whether the missing Historian is trapped in the area containing a <em>giant electromagnet</em> or <em>molten lava</em>. You'll need to make sure that for each of the five codes, you find the <em>shortest sequence</em> of button presses necessary.</p>
<p>The <em>complexity</em> of a single code (like <code>029A</code>) is equal to the result of multiplying these two values:</p>
<ul>
<li>The <em>length of the shortest sequence</em> of button presses you need to type on your directional keypad in order to cause the code to be typed on the numeric keypad; for <code>029A</code>, this would be <code>68</code>.</li>
<li>The <em>numeric part of the code</em> (ignoring leading zeroes); for <code>029A</code>, this would be <code>29</code>.</li>
</ul>
<p>In the above example, complexity of the five codes can be found by calculating <code>68 * 29</code>, <code>60 * 980</code>, <code>68 * 179</code>, <code>64 * 456</code>, and <code>64 * 379</code>. Adding these together produces <code><em>126384</em></code>.</p>
<p>Find the fewest number of button presses you'll need to perform in order to cause the robot in front of the door to type each code. <em>What is the sum of the complexities of the five codes on your list?</em></p>
</article>


In [ ]:
from collections import deque
from collections.abc import Iterator
from functools import cache
from itertools import chain
from pprint import pprint
from tabulate import tabulate


tests = [
    {
        "name": "Example",
        "s": """
            029A
            980A
            179A
            456A
            379A
        """,
        "expected": 126384,
    },
]

Cell = tuple[int, int]


class Pad(Str):
    def __init__(self, pad: list[list[int | chr]]) -> None:
        super().__init__()
        self.pad = pad
        self.inv = {
            pad[r][c]: (r, c)
            for r, c in product(range(len(pad)), range(len(pad[0])))
            if pad[r][c] != " "
        }
        self.paths = self._all_paths()

    def _all_paths(self) -> dict[tuple[tuple[Cell, Cell], list[str]]]:
        rows, cols = len(self.pad), len(self.pad[0])
        paths = defaultdict(list)
        dirs_to_str = {(-1, 0): "^", (0, 1): ">", (1, 0): "v", (0, -1): "<"}

        queue = deque(
            [
                (r, c, [])
                for r, c in product(range(rows), range(cols))
                if self.pad[r][c] != " "
            ]
        )

        while queue:
            r, c, path = queue.popleft()

            if not (0 <= r < rows and 0 <= c < cols):
                continue

            if self.pad[r][c] == " ":
                continue

            if (r, c) in path:
                continue

            path = path + [(r, c)]

            if len(path) > 1:
                r0, c0 = path[0]
                if len(path) - 1 > abs(r0 - r) + abs(c0 - c):
                    continue

                paths[(path[0], path[-1])].append(path)

            queue.extend(
                (r + dr, c + dc, path) for dr, dc in ((-1, 0), (0, 1), (1, 0), (0, -1))
            )

        return {
            k: [
                "".join(
                    dirs_to_str[(rr - lr, rc - lc)]
                    for (lr, lc), (rr, rc) in zip(vv[:-1], vv[1:])
                )
                for vv in v
            ]
            for k, v in paths.items()
        }

    @cache
    def all_possible_inputs(self, out: str) -> list[str]:
        @cache
        def dfs(fr: tuple[int, int], out: str) -> list[str]:
            if not out:
                return [""]

            out1 = out[1:]
            to = self.inv[out[0]]

            result = []

            for path in self.paths.get((fr, to), [""]):
                for res in dfs(to, out1):
                    result.append(f"{path}A{res}")

            return result

        return dfs(self.inv["A"], out)

    def push_pad(self, inp: str, debug=False) -> str:
        row, col = self.inv["A"]
        out = []

        if debug:
            print(f"start={type(self).__name__}.push_pad({inp=})")
            print(f"{row,col=}")
            print(self.str(row, col))

        for ch in inp:
            if ch == "A":
                out.append(self.pad[row][col])
            elif ch == "^":
                row -= 1
            elif ch == "v":
                row += 1
            elif ch == "<":
                col -= 1
            elif ch == ">":
                col += 1

            if debug:
                print(self.str(row, col))
                print(f"{ch=}, {out=}")

        return "".join(out)

    def str(self, row: int, col: int) -> str:
        pad = [
            [
                (
                    hexcolor_str("#ff0000", ch)
                    if r == row and c == col
                    else hexcolor_str("#0080ff", ch)
                )
                for c, ch in enumerate(rr)
            ]
            for r, rr in enumerate(self.pad)
        ]
        return tabulate(pad, tablefmt="rounded_grid")


class NumericalKeyPad(Pad):
    def __init__(self) -> None:
        super().__init__(
            [
                "789",
                "456",
                "123",
                " 0A",
            ]
        )


class DirectionalKeyPad(Pad):
    def __init__(self) -> None:
        super().__init__(
            [
                " ^A",
                "<v>",
            ]
        )


def complexity_of_codes(s: str, debug=False) -> int:
    nk = NumericalKeyPad()
    dk = DirectionalKeyPad()

    count = 0

    for code in re.findall(f"[A0-9]+", s):
        numeric_part = int(code[:-1])
        out1 = nk.all_possible_inputs(code)
        out2 = chain.from_iterable(dk.all_possible_inputs(out) for out in out1)
        out3 = chain.from_iterable(dk.all_possible_inputs(out) for out in out2)
        count += numeric_part * min(map(len, out3))
    return count


@test(tests=tests[:])
def partI_test(s: str) -> int:
    return complexity_of_codes(s, True)


Test Example passed, for partI_test.
Success


In [3]:
with open("../input/day21.txt") as f:
    puzzle = f.read()

print(f"Part I: { complexity_of_codes(puzzle) }")

Part I: 177814


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>177814</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Just as the missing Historian is released, The Historians realize that a <em>second</em> member of their search party has also been missing <span title="bum bum BUUUUUM">this entire time</span>!</p>
<p>A quick life-form scan reveals the Historian is also trapped in a locked area of the ship. Due to a variety of hazards, robots are once again dispatched, forming another chain of remote control keypads managing robotic-arm-wielding robots.</p>
<p>This time, many more robots are involved. In summary, there are the following keypads:</p>
<ul>
<li>One directional keypad that <em>you</em> are using.</li>
<li><em>25</em> directional keypads that <em>robots</em> are using.</li>
<li>One numeric keypad (on a door) that a <em>robot</em> is using.</li>
</ul>
<p>The keypads form a chain, just like before: your directional keypad controls a robot which is typing on a directional keypad which controls a robot which is typing on a directional keypad... and so on, ending with the robot which is typing on the numeric keypad.</p>
<p>The door codes are the same this time around; only the number of robots and directional keypads has changed.</p>
<p>Find the fewest number of button presses you'll need to perform in order to cause the robot in front of the door to type each code. <em>What is the sum of the complexities of the five codes on your list?</em></p>
</article>


In [ ]:
tests = [
    {
        "name": "Example",
        "s": """
            029A
            980A
            179A
            456A
            379A
        """,
        "expected": 126384,
    },
]

Cell = tuple[int, int]


class PadII(Pad): ...


class NumericalKeyPad(PadII):
    def __init__(self) -> None:
        super().__init__(
            [
                "789",
                "456",
                "123",
                " 0A",
            ]
        )


class DirectionalKeyPad(PadII):
    def __init__(self) -> None:
        super().__init__(
            [
                " ^A",
                "<v>",
            ]
        )


@test(tests=tests[:])
def partI_test(s: str) -> int:
    return complexity_of_codes(s, True)

720

<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>202972175280682</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
